In [ ]:
import pandas as pd
import os
import cv2
import numpy as np
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Dataset

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/rehab-ai-data/KiMoRe_final/KiMoRe_data_movenet.csv", index_col=False).drop('Unnamed: 0', axis=1)

In [ ]:
df.head()

,ID,exercise,video,joint_positions,clinical_score,#frames
0,P_ID11,Es2,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,12.000000,425
1,P_ID11,Es3,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,12.666667,394
2,P_ID11,Es1,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,14.666667,529
3,P_ID11,Es4,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,15.333333,363
4,P_ID11,Es5,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,12.666667,438


# KIMORE Features Extraction Functions

In [ ]:
def get_joint_pair(df, joint_name):
  return [df[f"{joint_name}_x"], df[f"{joint_name}_y"]]

In [ ]:
def calculate_angle(first, middle, end):
    first = np.array(first)
    middle = np.array(middle)
    end = np.array(end)

    radians = np.arctan2(end[1]-middle[1], end[0]-middle[0]) - np.arctan2(first[1]-middle[1], first[0]-middle[0])
    angle = np.abs(radians*180.0/np.pi)

    angles = [360-a if a > 180 else a for a in angle]
    return angles

In [ ]:
def calculate_distance(pair1, pair2):
  pair1_x, pair1_y = pair1
  pair2_x, pair2_y = pair2
  return pd.Series([math.dist([x1, y1], [x2, y2]) for x1, y1, x2, y2 in zip(pair1_x, pair1_y, pair2_x, pair2_y)])

In [ ]:
def get_es1_features(df):
  features_df = pd.DataFrame()

  features_df["left_arm_torso_angle"] = calculate_angle(get_joint_pair(df, "left_elbow"),
                                                   get_joint_pair(df, "left_shoulder"),
                                                   get_joint_pair(df, "left_hip"))

  features_df["right_arm_torso_angle"] = calculate_angle(get_joint_pair(df, "right_elbow"),
                                                   get_joint_pair(df, "right_shoulder"),
                                                   get_joint_pair(df, "right_hip"))

  features_df["left_elbow_extension_angle"] = calculate_angle(get_joint_pair(df, "left_shoulder"),
                                                     get_joint_pair(df, "left_elbow"),
                                                     get_joint_pair(df, "left_wrist"))

  features_df["right_elbow_extension_angle"] = calculate_angle(get_joint_pair(df, "right_shoulder"),
                                                     get_joint_pair(df, "right_elbow"),
                                                     get_joint_pair(df, "right_wrist"))

  features_df["left_knee_extension_angle"] = calculate_angle(get_joint_pair(df, "left_hip"),
                                                    get_joint_pair(df, "left_knee"),
                                                    get_joint_pair(df, "left_ankle"))

  features_df["right_knee_extension_angle"] = calculate_angle(get_joint_pair(df, "right_hip"),
                                                    get_joint_pair(df, "right_knee"),
                                                    get_joint_pair(df, "right_ankle"))

  mid_hip_point = [(get_joint_pair(df, "left_hip")[0] + get_joint_pair(df, "right_hip")[0])/2, (get_joint_pair(df, "left_hip")[1] + get_joint_pair(df, "right_hip")[1])/2]
  features_df["hip_angle"] = calculate_angle(get_joint_pair(df, "left_hip"),
                                        mid_hip_point,
                                        get_joint_pair(df, "right_hip"))

  features_df["hands_dist"] = calculate_distance(get_joint_pair(df, "left_wrist"), get_joint_pair(df, "right_wrist"))

  features_df["ankle_dist"] = calculate_distance(get_joint_pair(df, "left_ankle"), get_joint_pair(df, "right_ankle"))

  return features_df


def get_es2_features(df):
  features_df = pd.DataFrame()

  features_df["left_elbow_extension_angle"] = calculate_angle(get_joint_pair(df, "left_shoulder"),
                                                     get_joint_pair(df, "left_elbow"),
                                                     get_joint_pair(df, "left_wrist"))

  features_df["right_elbow_extension_angle"] = calculate_angle(get_joint_pair(df, "right_shoulder"),
                                                     get_joint_pair(df, "right_elbow"),
                                                     get_joint_pair(df, "right_wrist"))

  features_df["left_knee_extension_angle"] = calculate_angle(get_joint_pair(df, "left_hip"),
                                                    get_joint_pair(df, "left_knee"),
                                                    get_joint_pair(df, "left_ankle"))

  features_df["right_knee_extension_angle"] = calculate_angle(get_joint_pair(df, "right_hip"),
                                                    get_joint_pair(df, "right_knee"),
                                                    get_joint_pair(df, "right_ankle"))

  mid_hip_point = [(get_joint_pair(df, "left_hip")[0] + get_joint_pair(df, "right_hip")[0])/2, (get_joint_pair(df, "left_hip")[1] + get_joint_pair(df, "right_hip")[1])/2]
  features_df["hip_angle"] = calculate_angle(get_joint_pair(df, "left_hip"),
                                        mid_hip_point,
                                        get_joint_pair(df, "right_hip"))

  features_df["hands_dist"] = calculate_distance(get_joint_pair(df, "left_wrist"), get_joint_pair(df, "right_wrist"))

  features_df["shoulder_dist"] = calculate_distance(get_joint_pair(df, "left_shoulder"), get_joint_pair(df, "right_shoulder"))

  features_df["left_shoulder_wrist_vert_dist"] = np.abs(df["left_shoulder_y"] - df["left_wrist_y"])
  features_df["right_shoulder_wrist_vert_dist"] = np.abs(df["right_shoulder_y"] - df["right_wrist_y"])

  return features_df

def get_es3_features(df):
  features_df = pd.DataFrame()

  features_df["elbows_horiz_dist"] = np.abs(df["left_elbow_x"] - df["right_elbow_x"])

  features_df["left_elbow_extension_angle"] = calculate_angle(get_joint_pair(df, "left_shoulder"),
                                                     get_joint_pair(df, "left_elbow"),
                                                     get_joint_pair(df, "left_wrist"))

  features_df["right_elbow_extension_angle"] = calculate_angle(get_joint_pair(df, "right_shoulder"),
                                                     get_joint_pair(df, "right_elbow"),
                                                     get_joint_pair(df, "right_wrist"))

  features_df["left_knee_extension_angle"] = calculate_angle(get_joint_pair(df, "left_hip"),
                                                    get_joint_pair(df, "left_knee"),
                                                    get_joint_pair(df, "left_ankle"))

  features_df["right_knee_extension_angle"] = calculate_angle(get_joint_pair(df, "right_hip"),
                                                    get_joint_pair(df, "right_knee"),
                                                    get_joint_pair(df, "right_ankle"))

  features_df["left_shoulder_extension_angle"] = calculate_angle(get_joint_pair(df, "left_elbow"),
                                                    get_joint_pair(df, "left_shoulder"),
                                                    get_joint_pair(df, "right_shoulder"))

  features_df["right_shoulder_extension_angle"] = calculate_angle(get_joint_pair(df, "right_elbow"),
                                                    get_joint_pair(df, "right_shoulder"),
                                                    get_joint_pair(df, "left_shoulder"))

  mid_hip_point = [(get_joint_pair(df, "left_hip")[0] + get_joint_pair(df, "right_hip")[0])/2, (get_joint_pair(df, "left_hip")[1] + get_joint_pair(df, "right_hip")[1])/2]
  features_df["hip_angle"] = calculate_angle(get_joint_pair(df, "left_hip"),
                                        mid_hip_point,
                                        get_joint_pair(df, "right_hip"))

  features_df["hands_dist"] = calculate_distance(get_joint_pair(df, "left_wrist"), get_joint_pair(df, "right_wrist"))

  features_df["shoulder_dist"] = calculate_distance(get_joint_pair(df, "left_shoulder"), get_joint_pair(df, "right_shoulder"))

  features_df["hip_dist"] = calculate_distance(get_joint_pair(df, "left_hip"), get_joint_pair(df, "right_hip"))

  features_df["left_shoulder_wrist_vert_dist"] = np.abs(df["left_shoulder_y"] - df["left_wrist_y"])
  features_df["right_shoulder_wrist_vert_dist"] = np.abs(df["right_shoulder_y"] - df["right_wrist_y"])

  return features_df


def get_es4_features(df):
  features_df = pd.DataFrame()

  features_df["left_elbow_extension_angle"] = calculate_angle(get_joint_pair(df, "left_shoulder"),
                                                     get_joint_pair(df, "left_elbow"),
                                                     get_joint_pair(df, "left_wrist"))

  features_df["right_elbow_extension_angle"] = calculate_angle(get_joint_pair(df, "right_shoulder"),
                                                     get_joint_pair(df, "right_elbow"),
                                                     get_joint_pair(df, "right_wrist"))

  features_df["left_knee_extension_angle"] = calculate_angle(get_joint_pair(df, "left_hip"),
                                                    get_joint_pair(df, "left_knee"),
                                                    get_joint_pair(df, "left_ankle"))

  features_df["right_knee_extension_angle"] = calculate_angle(get_joint_pair(df, "right_hip"),
                                                    get_joint_pair(df, "right_knee"),
                                                    get_joint_pair(df, "right_ankle"))

  features_df["shoulder_dist"] = calculate_distance(get_joint_pair(df, "left_shoulder"), get_joint_pair(df, "right_shoulder"))

  features_df["hip_dist"] = calculate_distance(get_joint_pair(df, "left_hip"), get_joint_pair(df, "right_hip"))

  return features_df


def get_es5_features(df):
  features_df = pd.DataFrame()

  features_df["left_knee_extension_angle"] = calculate_angle(get_joint_pair(df, "left_hip"),
                                                    get_joint_pair(df, "left_knee"),
                                                    get_joint_pair(df, "left_ankle"))

  features_df["right_knee_extension_angle"] = calculate_angle(get_joint_pair(df, "right_hip"),
                                                    get_joint_pair(df, "right_knee"),
                                                    get_joint_pair(df, "right_ankle"))

  features_df["hands_dist"] = calculate_distance(get_joint_pair(df, "left_wrist"), get_joint_pair(df, "right_wrist"))

  features_df["shoulder_dist"] = calculate_distance(get_joint_pair(df, "left_shoulder"), get_joint_pair(df, "right_shoulder"))

  features_df["hip_dist"] = calculate_distance(get_joint_pair(df, "left_hip"), get_joint_pair(df, "right_hip"))

  features_df["knee_dist"] = calculate_distance(get_joint_pair(df, "left_knee"), get_joint_pair(df, "right_knee"))

  features_df["ankle_dist"] = calculate_distance(get_joint_pair(df, "left_ankle"), get_joint_pair(df, "right_ankle"))

  features_df["left_shoulder_wrist_dist"] = calculate_distance(get_joint_pair(df, "left_shoulder"), get_joint_pair(df, "left_wrist"))
  features_df["right_shoulder_wrist_dist"] = calculate_distance(get_joint_pair(df, "right_shoulder"), get_joint_pair(df, "right_wrist"))

  return features_df

# Test the Functions

In [ ]:
sample_es1_csv = df[(df["ID"] == "E_ID15") & (df["exercise"] == "Es1")]["joint_positions"].values[0]
sample_es2_csv = df[(df["ID"] == "E_ID12") & (df["exercise"] == "Es2")]["joint_positions"].values[0]
sample_es3_csv = df[(df["ID"] == "E_ID12") & (df["exercise"] == "Es3")]["joint_positions"].values[0]
sample_es4_csv = df[(df["ID"] == "E_ID1") & (df["exercise"] == "Es4")]["joint_positions"].values[0]
sample_es5_csv = df[(df["ID"] == "E_ID1") & (df["exercise"] == "Es5")]["joint_positions"].values[0]

In [ ]:
es1_df = pd.read_csv(sample_es1_csv)
es2_df = pd.read_csv(sample_es2_csv)
es3_df = pd.read_csv(sample_es3_csv)
es4_df = pd.read_csv(sample_es4_csv)
es5_df = pd.read_csv(sample_es5_csv)

In [ ]:
es1_kimore_features = get_es1_features(es1_df)
es1_kimore_features.head()

,left_arm_torso_angle,right_arm_torso_angle,left_elbow_extension_angle,right_elbow_extension_angle,left_knee_extension_angle,right_knee_extension_angle,hip_angle,hands_dist,ankle_dist
0,14.330966,3.493185,175.810762,148.839548,175.503716,176.210456,180.0,0.007766,0.009974
1,10.890182,11.116299,174.091247,179.916705,179.482406,179.834869,180.0,0.129922,0.063665
2,10.655145,11.119946,173.675025,179.223380,178.534108,179.606737,180.0,0.132111,0.062650
3,10.829913,11.473507,173.154919,179.453744,179.918393,178.416733,180.0,0.132841,0.063439
4,10.432295,11.306615,172.779489,179.718302,179.563459,179.224962,180.0,0.131670,0.062243


In [ ]:
es2_kimore_features = get_es2_features(es2_df)
es2_kimore_features.head()

,left_elbow_extension_angle,right_elbow_extension_angle,left_knee_extension_angle,right_knee_extension_angle,hip_angle,hands_dist,shoulder_dist,left_shoulder_wrist_vert_dist,right_shoulder_wrist_vert_dist
0,155.393909,163.940419,176.927707,179.296316,180.0,0.091229,0.064310,0.126427,0.141040
1,178.390445,179.746161,176.167052,176.827579,180.0,0.096596,0.068864,0.189413,0.184384
2,177.066967,179.635208,175.249032,177.094424,180.0,0.099634,0.068067,0.197624,0.192286
3,177.829437,178.848155,177.013341,175.081152,180.0,0.098019,0.065886,0.192392,0.195092
4,177.680934,179.736735,176.723954,176.308044,180.0,0.097035,0.066475,0.196510,0.194642


In [ ]:
es3_kimore_features = get_es3_features(es3_df)
es3_kimore_features.head()

,elbows_horiz_dist,left_elbow_extension_angle,right_elbow_extension_angle,left_knee_extension_angle,right_knee_extension_angle,left_shoulder_extension_angle,right_shoulder_extension_angle,hip_angle,hands_dist,shoulder_dist,hip_dist,left_shoulder_wrist_vert_dist,right_shoulder_wrist_vert_dist
0,0.103262,169.327898,169.545749,176.596389,177.557962,94.104428,108.336919,180.0,0.106187,0.075921,0.038671,0.169310,0.158390
1,0.098104,175.391050,179.953326,175.927570,176.722068,96.388402,93.722823,180.0,0.106578,0.076421,0.047227,0.209399,0.205626
2,0.096833,177.304004,177.939368,176.913960,174.815745,95.691101,95.656901,180.0,0.107280,0.073220,0.045609,0.210640,0.203942
3,0.099585,174.483732,176.538743,176.868026,175.010264,95.978202,96.450531,180.0,0.105755,0.073419,0.045135,0.216915,0.206032
4,0.099226,176.909406,174.736249,177.982148,175.004129,96.920912,95.055702,180.0,0.105531,0.074250,0.045798,0.216872,0.210264


In [ ]:
es4_kimore_features = get_es4_features(es4_df)
es4_kimore_features.head()

,left_elbow_extension_angle,right_elbow_extension_angle,left_knee_extension_angle,right_knee_extension_angle,shoulder_dist,hip_dist
0,137.048249,174.049160,119.676085,116.935714,0.090570,0.057237
1,110.475728,116.531713,179.712821,175.804830,0.068354,0.043287
2,106.527069,114.597275,178.611754,175.836680,0.072463,0.043338
3,113.346031,115.993588,178.500319,176.983860,0.075690,0.042641
4,119.125273,118.963380,178.442177,176.523395,0.073648,0.041647


In [ ]:
es5_kimore_features = get_es5_features(es5_df)
es5_kimore_features.head()

,left_knee_extension_angle,right_knee_extension_angle,hands_dist,shoulder_dist,hip_dist,knee_dist,ankle_dist,left_shoulder_wrist_dist,right_shoulder_wrist_dist
0,173.943330,174.658208,0.092314,0.056443,0.026156,0.016049,0.036713,0.171170,0.160803
1,178.117137,179.326536,0.132519,0.069225,0.043954,0.053946,0.056479,0.195569,0.188672
2,177.139307,178.975396,0.134016,0.070031,0.043465,0.055385,0.056656,0.192260,0.190057
3,178.058307,179.908371,0.135149,0.069864,0.043513,0.053227,0.056266,0.192656,0.189342
4,178.201344,179.971559,0.135859,0.069565,0.042779,0.052690,0.056246,0.193186,0.191407


# Create the Joint Features Dataset

In [ ]:
target_path = '/content/drive/MyDrive/rehab-ai-data/KiMoRe_final'

In [ ]:
for index, row in df.iterrows():
    exercise = row["exercise"]
    joint_positions_path = row['joint_positions']
    if joint_positions_path is np.NAN:
      row["joint_features"] = np.NaN
      continue

    joint_positions_file_name = os.path.basename(joint_positions_path)[:-4] # removes .csv extension too
    joint_positions_dir = os.path.dirname(joint_positions_path)
    joint_positions_data = pd.read_csv(joint_positions_path)

    joint_features_data = None
    if exercise == "Es1":
      joint_features_data = get_es1_features(joint_positions_data)
    elif exercise == "Es2":
      joint_features_data = get_es2_features(joint_positions_data)
    elif exercise == "Es3":
      joint_features_data = get_es3_features(joint_positions_data)
    elif exercise == "Es4":
      joint_features_data = get_es4_features(joint_positions_data)
    elif exercise == "Es5":
      joint_features_data = get_es5_features(joint_positions_data)

    joint_features_path = f"{joint_positions_dir}/{joint_positions_file_name}_features.csv"
    df.loc[index, "joint_features"] = joint_features_path
    joint_features_data.to_csv(joint_features_path, index=False)
df = df[['ID', 'exercise', 'video', 'joint_positions', 'joint_features', 'clinical_score', '#frames']]

In [ ]:
df.head()

,ID,exercise,video,joint_positions,joint_features,clinical_score,#frames
0,P_ID11,Es2,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,12.000000,425
1,P_ID11,Es3,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,12.666667,394
2,P_ID11,Es1,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,14.666667,529
3,P_ID11,Es4,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,15.333333,363
4,P_ID11,Es5,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,/content/drive/MyDrive/rehab-ai-data/KiMoRe_rg...,12.666667,438


In [ ]:
df.to_csv(f"{target_path}/KiMoRe_data_movenet_features.csv", index=False)